# AM2 Model Predictive Control (MPC) Tutorial

This notebook demonstrates how to implement a Model Predictive Controller (MPC) for the AM2 anaerobic digestion model using the `openad_lib` library and `do-mpc`.

## Objective
The goal is to maximize biogas production ($Q$) by manipulating the dilution rate ($D$), subject to process constraints and disturbances in inlet substrate concentration ($S_{1in}$).


In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt

# Add src to path if running locally
sys.path.append(os.path.join(os.getcwd(), '..', 'src'))

from openad_lib.models.mechanistic import AM2Parameters
from openad_lib.control import AM2MPC

## 1. Setup Controller

We initialize the `AM2MPC` controller with default parameters. The controller uses `do-mpc` under the hood to formulate the optimization problem.

In [ ]:
# Initialize parameters and controller
params = AM2Parameters()
controller = AM2MPC(params)

# Configure MPC
sampling_time = 1.0  # days
horizon = 10         # prediction horizon steps

controller.setup_controller(
    sampling_time=sampling_time,
    horizon=horizon,
    objective_type='maximize_biogas',
    D_max=0.5
)

# Setup Simulator for validation
controller.setup_simulator(sampling_time=sampling_time)

## 2. define Simulation Scenario

We simulate a 50-day operation where the inlet substrate concentration ($S_{1in}$) varies sinusoidally, simulating a fluctuating feedstock.

In [ ]:
# Initial State: [S1, X1, S2, X2]
x0 = np.array([5.0, 1.0, 10.0, 1.0])
controller.set_initial_state(x0)

# Simulation Loop
n_days = 50

# Storage
time = []
D_log = []
S1_log = []
S2_log = []
Q_log = []
S1in_log = []

t_current = 0.0

print("Starting simulation...")

for k in range(n_days):
    # Time-varying disturbance
    S1in_k = 15.0 + 5.0 * np.sin(2 * np.pi * k / 20)
    pH_k = 7.0 # Constant optimal pH
    
    # Run MPC Step
    u_opt, y_next = controller.run_step(S1in_val=S1in_k, pH_val=pH_k)
    
    # Calculate Biogas (Q) for visualization
    # Extract states
    S2_curr = float(y_next[2])
    X2_curr = float(y_next[3])
    
    # Calculate kinetics
    mu2_base = params.m2 * (S2_curr / ((S2_curr**2)/params.Ki + S2_curr + params.K2))
    pH_factor = np.exp(-4 * ((pH_k - params.pHH)/(params.pHH - params.pHL))**2)
    mu2 = mu2_base * (1.0 - pH_factor)
    Q_curr = float(params.k6 * mu2 * X2_curr * params.c)
    
    # Store data
    time.append(t_current)
    D_log.append(u_opt)
    S1_log.append(float(y_next[0]))
    S2_log.append(float(y_next[2]))
    Q_log.append(Q_curr)
    S1in_log.append(S1in_k)
    
    t_current += sampling_time

print("Simulation complete.")

## 3. Results Visualization

The plots below show the system response. The MPC adjusts the dilution rate ($D$) to maximize biogas production ($Q$) while managing the fluctuating potential of the inlet substrate.

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(10, 12), sharex=True)

# 1. States
ax = axes[0]
ax.plot(time, S1_log, label='S1 (Substrate)')
ax.plot(time, S2_log, label='S2 (VFA)')
ax.set_ylabel('Concentration [g/L]')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('State Trajectories')

# 2. Biogas
ax = axes[1]
ax.plot(time, Q_log, 'g', linewidth=2, label='Biogas Q')
ax.set_ylabel('Q [L/d]')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_title('Biogas Production')

# 3. Control Input & Disturbance
ax = axes[2]
ax.step(time, D_log, 'r-', where='post', label='Dilution Rate D')
ax2 = ax.twinx()
ax2.plot(time, S1in_log, 'k--', alpha=0.5, label='Inlet S1in')
ax.set_ylabel('D [1/d]')
ax2.set_ylabel('S1in [g/L]')
ax.set_xlabel('Time [d]')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_title('Controls and Disturbances')

plt.tight_layout()
plt.show()